In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json


In [5]:

# URL of the FAQ page
url = 'https://www.getirarac.com/tr/sikca-sorulan-sorular#genel-bilgiler-kullanim-ve-kosullar'

# Send a request to get the HTML content of the page
response = requests.get(url)
html_content = response.content

In [6]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

In [7]:
# Function to extract FAQs
def extract_faqs(soup):
    faqs = []

    # Loop through the FAQ questions and answers
    questions = soup.find_all('div', class_='question')
    answers = soup.find_all('div', class_='answer')

    for question, answer in zip(questions, answers):
        # Extract the question text
        question_text = question.find('span').get_text(strip=True) if question.find('span') else "No question"

        # Extract the answer text (cleaning up HTML tags)
        answer_text = answer.get_text(strip=True)

        # Try to infer category based on parent section or other clues
        category = "General"  # Default category (can be improved based on structure)
        category_tag = question.find_previous('h2')  # Assuming category headings are in <h2> tags
        if category_tag:
            category = category_tag.get_text(strip=True)

        # Append the FAQ to the list
        faqs.append({
            'category': category,
            'question': question_text,
            'answer': answer_text
        })

    return faqs

In [8]:
# Extract FAQs
faqs = extract_faqs(soup)

# Save to CSV using pandas
df = pd.DataFrame(faqs)
df.to_csv('faq_data.csv', index=False)

# Save to JSON
with open('faq_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(faqs, json_file, ensure_ascii=False, indent=4)

print("FAQ data saved to 'faq_data.csv' and 'faq_data.json'")

FAQ data saved to 'faq_data.csv' and 'faq_data.json'


In [10]:
data = pd.read_csv('faq_data.csv')
data.head(4)

,category,question,answer
0,General,Güvenli Sürüş Puanı nedir?,Güvenli sürüşün takipçisiyiz! Araçlarımızdaki ...
1,General,GetirAraç nedir?,"GetirAraç saatlik, dakikalık veya günlük araç ..."
2,General,Günlük kullanıma geçiş nedir?,10 saati aşan kullanımların otomatik olarak gü...
3,General,Saatlik / günlük paketler nedir?,"Saatlik ve günlük paketler, planı az çok belli..."
